### Purpose of notebook
The purpose of this notebook is to pass the YouTube ASL videos through the MoviNet backbone and then save its output back to S3

In [1]:
# !git clone https://github.com/tensorflow/models.git
# !pip install s3fs
# !pip install boto3

In [2]:
# Let's not show unnecessary warnings etc
import warnings
warnings.filterwarnings('ignore')

In [3]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [4]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [5]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')

In [6]:
# Import all required libraries. Keep adding here as you code
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

tf.keras.backend.clear_session()

2023-11-21 04:20:05.747744: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-21 04:20:05.747795: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-21 04:20:05.747812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-21 04:20:05.754195: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-21 04:20:06.675882: W tensorflow/compiler/

In [7]:
# Set working directory
%cd /content/models

[Errno 2] No such file or directory: '/content/models'
/home/ec2-user/ASL-Translator/modeling


In [8]:
!pwd

/home/ec2-user/ASL-Translator/modeling


In [9]:
# Define the model you are going to use
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [10]:
# Get the kinetics-600 action labels
KINETICS_URL = "/home/ec2-user/models/official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [11]:
# !pip install tf_keras
# !pip install gin-config
# !pip install immutabledict

In [12]:
# Create the backbone using the Movinet model
from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)

2023-11-21 04:20:08.930679: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 04:20:08.951675: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 04:20:08.953730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
# Convert the modified model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(backbone)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS     # Enable TF Select ops.
]
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpoa80aws4/assets


INFO:tensorflow:Assets written to: /tmp/tmpoa80aws4/assets
2023-11-21 04:22:41.885951: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-21 04:22:41.886000: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-21 04:22:41.886617: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpoa80aws4
2023-11-21 04:22:42.066794: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-21 04:22:42.066833: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpoa80aws4
2023-11-21 04:22:42.466955: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-21 04:22:42.614706: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-21 04:22:44.120924: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [14]:
# Save the TFLite model to a file
with open('backbone_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [15]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='backbone_model.tflite')

INFO: Created TensorFlow Lite delegate for select TF ops.
2023-11-21 04:23:03.395155: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 04:23:03.396580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-21 04:23:03.397272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.c

In [17]:
# Get input details without allocating tensors
input_details = interpreter.get_input_details()

# Print the expected input shape
print("Expected input shape:", input_details[0]['shape'])

Expected input shape: [1 1 1 1 3]


In [31]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [22]:
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [32]:
# Get a list of all the numpy files from S3

paginator = s3.get_paginator('list_objects_v2')
video_files = []
for page in paginator.paginate(Bucket = bucket_name, Prefix = 'youtube-asl/1000-samples/numpy_files/'):
    video_files.extend(content['Key'] for content in page.get('Contents',[]) if content['Key'].endswith(('.npy')))
print(len(video_files))

59042


In [33]:
# Add the S3 prefix
video_files = ['s3://asl-capstone/'+x for x in video_files]

In [37]:
# Let's break this into batches
video_files = video_files[:3]

In [38]:
iteration_2 = video_files #UPDATE HERE in 1000 video increments

In [39]:
print(len(iteration_2))

3


In [41]:
interpreter.resize_tensor_input(0, [1, 120, 256, 256, 3], strict=True)

In [42]:
for vid in iteration_2:
    filename = os.path.basename(vid)
    
    with fs.open(vid, "rb") as f:
        vid_file = np.load(f)
    vid_file = np.expand_dims(vid_file, axis=0)
    # Set the value of the input tensor
    interpreter.set_tensor(input_details[0]['index'], vid_file)

    # Run inference
    interpreter.invoke()
    
    # Extract the embeddings
    embeddings = interpreter.get_tensor(output_details[0]['index'])

    # Save the embeddings
    with fs.open(f"{movinet_uri+filename}", "wb") as f:
        np.save(f, embeddings)

ValueError: Cannot set tensor: Dimension mismatch. Got 34 but expected 120 for dimension 1 of input 0.

In [38]:
paginator = s3.get_paginator('list_objects_v2')
completed_files = []
for page in paginator.paginate(Bucket = bucket_name, Prefix = 'youtube-asl/1000-samples/movinet/backbone/'):
    completed_files.extend(content['Key'] for content in page.get('Contents',[]) if content['Key'].endswith(('.npy')))
print(len(completed_files))

3651


In [39]:
completed_files = [os.path.basename(x) for x in completed_files]

In [40]:
iteration_2 = [x for x in iteration_2 if os.path.basename(x) not in completed_files]
print(len(iteration_2))

200


In [41]:
# # Iteration batch
# iteration_1 = video_files[:5000]
# iteration_2 = video_files[5768:10000]
# iteration_3 = video_files[10001:15000]
# iteration_4 = video_files[15000:]

In [42]:
movinet_uri = 's3://asl-capstone/youtube-asl/1000-samples/movinet/backbone/'

In [43]:
def process_video(vid):
  # Add a try except
  try:
      # Generate video embeddings and store to CPU
      filename = os.path.basename(vid)
      with fs.open(vid,"rb") as f:
          vid_file = np.load(f)
      vid_file = np.expand_dims(vid_file, axis=0)
      embeddings = backbone(vid_file)[0]['block4_layer9'].cpu().numpy()
      with fs.open(f"{movinet_uri+filename}","wb") as f:
          np.save(f,embeddings)
  except:
      print(f"Error with processing {vid}")

In [44]:
import dask
from dask.distributed import Client, progress, as_completed

In [45]:
# Start a Dask client
client = Client(processes=False)
# Print the link to the dashboard
print("-------------------------------------------------------------")
print(client.dashboard_link)

-------------------------------------------------------------
http://172.31.13.102:42283/status


In [ ]:
# Use Dask's map function to apply the process_video function to each video
futures = client.map(process_video, iteration_2)

# Create a progress bar with tqdm
progress_bar = tqdm(total=len(iteration_2))

# Use as_completed to update the progress bar as futures complete
for future in as_completed(futures):
    result = future.result()  # This blocks until the future is complete
    progress_bar.update()

progress_bar.close()

# Wait for all futures to complete
client.gather(futures)

  0%|                                                                                                                                                                              | 0/200 [00:00<?, ?it/s]2023-11-20 03:17:05.853383: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3458031ad0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-20 03:17:05.853420: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-20 03:17:06.657220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8903
2023-11-20 03:17:07.660820: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Instructions for updating:
Use tf.identity with explicit device placement instead.


  8%|█████████████▏                                                                                                                                                       | 16/200 [03:11<15:19,  5.00s/it]